In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [4]:
RL = 'https://events.linuxfoundation.org/'
driver.get(RL)
try:
    driver.maximize_window()
except:
    pass

In [12]:
soup=bs(driver.page_source,'lxml')
cards=soup.find_all('div',{'class':'cell medium-6'})
len(cards)

0

In [10]:
linuxf=[]
for i in cards[66:]:
    ritle=' '.join(i.find('h4').text.split())
    link=i.find('a')['href']
    dat_dur=' '.join(i.find('p',{'class':re.compile('event-meta text.+')}).find('span').text.split())
    
    dat=dat_dur.replace('–','-')
    if '(' in dat:
        dat=dat.split('(')[0].strip()
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat
        #try:
#             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
#             mad=ma.group()

        if '-' in mad:
            st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        elif 'and'in mad:
            st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        elif '&' in mad:
            st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        else:
            st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        #print(date__,st,en)
        ####################
        if any(c.isalpha() for c in st)==False:
        #      print('leg')
            pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
            sapa=pa.group()
            start=st+' '+sapa
            end=en
        #                        02        April      2022
        elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
         #   print('awujale')
            start=st
            end=en

        #*                        April       02      2022
        elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
        #       print('ala')
            pa=re.search('(\d{4})',st).group()
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()
            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',en).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba     

        #                      02     2022
        elif re.search('\d{1,2}\s+\d{4}',en):
        #      print('kum')
            pa=re.search('(\d{4})',en).group()#, maxsplit=0)
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()

            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',st).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba

        #                      02         April
        elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
        #        print('is')
            pa=re.search('(\d{4})',en)#, maxsplit=0)
            sapa=pa.group()
            start=st+' '+sapa
            end=en

        #*                      April           02
        elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
        #      print('bad')
            pa=re.search('(\d{4})',en).group()#, maxsplit=0)
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()

            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',en).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba

        else:
        #     print('shik')
            start=end=''
        if start=='':
            start_date=end_date=''
        else:
            pick=[start, end]
            try:
                spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
            except:
                spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
            date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
            start_date=date_[0]
            end_date=date_[1]
    #**************
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            sha=location.split(',')
            try:
                gu(sha[-3])
            except:
                try:
                    gu(sha[-2])
                except:
                    try:
                        gu(sha[-1])
                    except Exception as e:
                        print(location, "; url didn't go through")
                        print(e)
                        return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")
    
    
    #************
    
    
    try:
        loc=' '.join(i.find('p',{'class':re.compile('event-meta text.+')}).find('span',{'class':'country'}).text.split()).replace('and','').replace('Irel','Ireland').strip()
    except:
        loc=''
        
    
    info=' '.join(i.find_all('p',{'class':re.compile('text-small small-margin-botto.+')})[1].text.split())
    
    driver.get(link)
    
    soupn=bs(driver.page_source,'lxml')
    
    try:
        title=' '.join(soupn.find('h1',{'class':'intro-heading heading'}).text.split())
    except:
        try:
            title=' '.join(soupn.find('h1',{'class':'show-for-sr'}).text.split())
        except:
            try:
                title=' '.join(soupn.find('h1',{'class':'content-wrapper'}).text.split())
            except:
                title=' '.join(soupn.find('span',{'itemprop':'title'}).text.split())
            
    try:
        
        event_info=' '.join(soupn.find('div',{'class':'entry-content'}).p.text.split())
    except:
        try:
            sha=' '.join(soupn.find('section',{'id':'dates'}).find('div',{'class':'row'}).text.split())
            event_info=' '.join(soupn.find('section',{'id':'dates'}).text.split()).replace(sha,'').strip()
        except:
            try:
                event_info=' '.join(soupn.find('div',{'class':'page-content'}).p.text.split())
            except:
                event_info=info
    try:
        if len(info)<75:
            try:
                event_info=' '.join(soupn.find('div',{'class':re.compile('wp-container-2 wp-block-group.+')}).text.split())
            except:
                try:
                    event_info=' '.join(soupn.find('div',{'class':re.compile('wp-container-3 wp-block-group.+')}).text.split())
                except:
                    try:
                        event_info=' '.join(soupn.find('div',{'class':re.compile('wp-container-1 wp-block-group.+')}).text.split())
                    except:
                        event_info=''
    except:
        event_info=info
    try:            
        sps=soupn.find_all('div',{'class':'wp-block-cgb-sponsors-block'})
    except:
        sps=soupn.find_all('div',{'class':'SponsorClass'})
    spon=[]
    for j in sps:
        sp=j.find('a')['href'].split('.')[1]
        if '/' in sp or 'net' in sp or 'com'in sp or 'org' in sp:
            pass
        else:
            spon.append(sp)
    if spon=='' or spon==[]:
        sponsor=''
    else:
        sponsor=json.dumps(spon)
    try:
        spk=soupn.find('section',{'class':re.compile('speakers-section.+')}).find_all('li')
    
        speak=[]
        for k in spk:
            try:
                na=' '.join(k.find('span',{'class':'name'}).text.split())
            except:
                na=' '.join(k.find('a',{'class':'name'}).text.split())
            try:
                tle=' '.join(k.find('span',{'class':'title'}).text.split())
            except:
                tle=' '.join(k.find('a',{'class':'title'}).text.split())
            ln=''
            toge={
                'name':na,
                'title':tle,
                'link':ln
            }
            speak.append(toge)
    except:
        speak=''
        
    if speak=='' or speak==[] or speak=='-' or speak==' ':
        Speakerlist=''
    else:
        Speakerlist=json.dumps(speak)
    ticket_list=''
    org_name='The Linux Foundation®'
    org_web='https://linuxfoundation.org/'
    org_pro='The Linux Foundation provides a neutral, trusted hub for developers to code, manage, and scale open technology projects.'
    logo=''    
    #sponsor=''
    agendalist=''
    type_=''
    category=''
    #Speakerlist='' 
    time_=''
    event_web=link
    mail_=json.dumps([' events@linuxfoundation.org'])
    
        
    if loc=='' or loc==' ' or loc=='-':
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        if ',' in loc:
            city=loc.split(',')[-2].strip()
            country=loc.split(',')[-1].strip()
        else:
            city=loc
            country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)
    
    linuxf.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])
print('Done')


Done


[['https://events.linuxfoundation.org/openjs-world/',
  'OpenJS World',
  '2022-06-06',
  '2022-06-10',
  '',
  'Learn and engage with leaders building modern applications at a massive scale. There will be something for everyone who loves JavaScript, including content around development, security, performance, automation & testing, and community building.',
  '',
  'The Linux Foundation provides a neutral, trusted hub for developers to code, manage, and scale open technology projects.',
  'The Linux Foundation®',
  'https://linuxfoundation.org/',
  '',
  '["ibm", "nearform", "techatbloomberg"]',
  '',
  '',
  '',
  'Austin',
  'United States',
  'Austin, United States',
  'https://events.linuxfoundation.org/openjs-world/',
  'https://www.google.com/maps/place/Austin,+TX,+USA/data=!4m2!3m1!1s0x8644b599a0cc032f:0x5d9b464bd469d57a?sa=X&ved=2ahUKEwjPn-DAtIb4AhVXPcAKHfmUDukQ8gF6BAhHEAE',
  '[" events@linuxfoundation.org"]',
  '[{"name": "Brian Behlendorf", "title": "General Manager, Open So

In [11]:
len(linuxf)

82

In [13]:
linuxf

[['https://events.linuxfoundation.org/openjs-world/',
  'OpenJS World',
  '2022-06-06',
  '2022-06-10',
  '',
  'Learn and engage with leaders building modern applications at a massive scale. There will be something for everyone who loves JavaScript, including content around development, security, performance, automation & testing, and community building.',
  '',
  'The Linux Foundation provides a neutral, trusted hub for developers to code, manage, and scale open technology projects.',
  'The Linux Foundation®',
  'https://linuxfoundation.org/',
  '',
  '["ibm", "nearform", "techatbloomberg"]',
  '',
  '',
  '',
  'Austin',
  'United States',
  'Austin, United States',
  'https://events.linuxfoundation.org/openjs-world/',
  'https://www.google.com/maps/place/Austin,+TX,+USA/data=!4m2!3m1!1s0x8644b599a0cc032f:0x5d9b464bd469d57a?sa=X&ved=2ahUKEwjPn-DAtIb4AhVXPcAKHfmUDukQ8gF6BAhHEAE',
  '[" events@linuxfoundation.org"]',
  '[{"name": "Brian Behlendorf", "title": "General Manager, Open So

In [14]:
linuxf_dict=linuxf


linuxf_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=linuxf_dict)
linuxf_df.to_csv("linuxf.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [8]:
# s=31
# for k in linuxf[70:80]:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[1])
#     s+=1

In [13]:
# for k in range(1,41):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(3)

In [9]:
# truce=[]
# for k in range(1,41):
#     driver.switch_to.window(driver.window_handles[k])
#     #time.sleep(1)
#     soupn=bs(driver.page_source,'lxml')
#     try:
#         title=' '.join(soupn.find('h1',{'class':'intro-heading heading'}).text.split())
#     except:
#         try:
#             title=' '.join(soupn.find('h1',{'class':'show-for-sr'}).text.split())
#         except:
#             try:
#                 title=' '.join(soupn.find('h1',{'class':'content-wrapper'}).text.split())
#             except:
#                 title=' '.join(soupn.find('span',{'itemprop':'title'}).text.split())
            
#     try:
        
#         info=' '.join(soupn.find('div',{'class':'entry-content'}).p.text.split())
#     except:
#         try:
#             sha=' '.join(soupn.find('section',{'id':'dates'}).find('div',{'class':'row'}).text.split())
#             info=' '.join(soupn.find('section',{'id':'dates'}).text.split()).replace(sha,'').strip()
#         except:
#             info=' '.join(soupn.find('div',{'class':'page-content'}).p.text.split())
#     if len(info)<75:
#         try:
#             info=' '.join(soupn.find('div',{'class':re.compile('wp-container-2 wp-block-group.+')}).text.split())
#         except:
#             try:
#                 info=' '.join(soupn.find('div',{'class':re.compile('wp-container-3 wp-block-group.+')}).text.split())
#             except:
#                 try:
#                     info=' '.join(soupn.find('div',{'class':re.compile('wp-container-1 wp-block-group.+')}).text.split())
#                 except:
#                     info=''
#     try:            
#         sps=soupn.find_all('div',{'class':'wp-block-cgb-sponsors-block'})
#     except:
#         sps=soupn.find_all('div',{'class':'SponsorClass'})
#     spon=[]
#     for j in sps:
#         sp=j.find('a')['href'].split('.')[1]
#         if '/' in sp or 'net' in sp or 'com'in sp or 'org' in sp:
#             pass
#         else:
#             spon.append(sp)
#     if spon=='' or spon==[]:
#         sponsor=''
#     else:
#         sponsor=json.dumps(spon)
#     try:
#         spk=soupn.find('section',{'class':re.compile('speakers-section.+')}).find_all('li')
    
#         speak=[]
#         for k in spk:
#             try:
#                 na=' '.join(k.find('span',{'class':'name'}).text.split())
#             except:
#                 na=' '.join(k.find('a',{'class':'name'}).text.split())
#             try:
#                 tle=' '.join(k.find('span',{'class':'title'}).text.split())
#             except:
#                 tle=' '.join(k.find('a',{'class':'title'}).text.split())
#             ln=''
#             toge={
#                 'name':na,
#                 'title':tle,
#                 'link':ln
#             }
#             speak.append(toge)
#     except:
#         speak=''
        
    
                
                
#     truce.append([title,info,sponsor,speak])
# truce

[['SODACODE',
  'Coding in open source projects is a great opportunity for a developer to become more proficient. SODACODE 2022 is the first coding event by SODA Foundation for beginners and professionals. From minor bug fixing to complex issues, and new feature development for SODA Projects, you will get to connect, network, and increase your visibility through working with experts in the industry. We have projects in multiple programming languages. Take your open source contributions to the next level or make your first contribution to open source! Submit Pull Requests (PR) or raise issues on GitHub. Enrich your open source portfolio through SODA Projects. What more, you can win amazing prizes worth USD 6,000+, contribution badges, and certifications. Selected participants will get a chance to share their experience and showcase their code in our final gala event of SODACODE 2022 with a large open source community of members and experts across the world. The Data & Storage Hackathon,